In [1]:
from keras.models import load_model
from collections import deque
import numpy as np
import pickle
import cv2

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
model = load_model('/content/drive/MyDrive/FinalCse499/RS50model.h5')
lb = pickle.loads(open('/content/drive/MyDrive/FinalCse499/modelRS50/VideoClassificationBinarizer.pickle','rb').read())
outputVideo = '/content/drive/MyDrive/FinalCse499/Video_Predict_Model_RS50/outputVideo/demo_output.mp4'
Queue = deque(maxlen=128)

In [5]:
print(lb)

['Boishak', 'Convocation', 'Meeting', 'Violence', 'Wedding']


In [6]:
import time

In [ ]:
capture_video = cv2.VideoCapture(r'3class.mp4')
writer = None
(width,height) = (None,None)
while True:
    (taken,frame) = capture_video.read()
    if not taken:
        break
    if width is None or height is None:
        width,height = frame.shape[:2]
    output = frame.copy()
    frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame,(128,128)).astype("float32")
    #frame -= mean
    preds = model.predict(np.expand_dims(frame,axis=0))[0]
    #print('PREd',preds)
    Queue.append(preds)
    results = np.array(Queue).mean(axis=0)
    #print('Result',results)
    i = np.argmax(results)
    #print("I",i)
    label = lb[i]
    #print('Label',label)
    text = "Involvement In:{}".format(label)
    cv2.putText(output,text,(45,60),cv2.FONT_HERSHEY_SIMPLEX,1.25,(255,255,10),5)
    
    if writer is None:
        sz = (int(capture_video.get(cv2.CAP_PROP_FRAME_WIDTH)),
        int(capture_video.get(cv2.CAP_PROP_FRAME_HEIGHT)))
        fourcc = cv2.VideoWriter_fourcc(*'mpeg')
        writer =  cv2.VideoWriter(outputVideo,fourcc,30,sz,True)
    writer.write(output)
    cv2.imshow("In progress",output)
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
writer.release()   
capture_video.release()

# Destroy all the windows
cv2.destroyAllWindows()